<a href="https://colab.research.google.com/github/pranavkantgaur/curves_and_surfaces/blob/master/curves_surfaces_monolithic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## What are curves and surfaces?

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate
import bezier  # Ensure you have the bezier library installed
from geomdl import BSpline

# Sample data points
x_data = np.array([0, 1, 2, 3])
y_data = np.array([0, 1, 4, 9])  # Quadratic function y = x^2

# Prepare a fine grid for plotting curves
x_fine = np.linspace(0, 3, 100)

# Linear Curve
y_linear = x_fine

# Quadratic Curve
y_quadratic = x_fine**2

# Cubic Curve
y_cubic = (x_fine**3) / 27  # Scale for better visualization

# Spline Curve
spline_tck = interpolate.splrep(x_data, y_data)
y_spline = interpolate.splev(x_fine, spline_tck)

# NURBS Curve (using geomdl)
nurbs_curve = BSpline.Curve()
nurbs_curve.degree = 3
nurbs_curve.ctrlpts = [[0, 0], [1, 1], [2, 4], [3, 1]]  # Control points for NURBS
nurbs_curve.knotvector = [0, 0, 0, 1, 1, 1]  # Example knot vector for a cubic NURBS
nurbs_points = nurbs_curve.evaluate(num_pts=100).pts
x_nurbs = nurbs_points[:, 0]
y_nurbs = nurbs_points[:, 1]

# Bézier Curve (using bezier library)
bezier_nodes = np.array([[0, 0], [1.5, 2.5], [2.5, 2], [3, 0]])  # Control points for Bézier curve
bezier_curve = bezier.Curve(bezier_nodes.T, degree=len(bezier_nodes) - 1)
bezier_points = bezier_curve.evaluate_list(np.linspace(0.0, 1.0, num=100)).T
x_bezier = bezier_points[:, 0]
y_bezier = bezier_points[:, 1]

# Plotting all curves
plt.figure(figsize=(12, 8))

plt.subplot(2, 3, 1)
plt.plot(x_fine, y_linear, label='Linear')
plt.title('Linear Curve')
plt.xlabel('X')
plt.ylabel('Y')
plt.grid()
plt.legend()

plt.subplot(2, 3, 2)
plt.plot(x_fine, y_quadratic, label='Quadratic', color='orange')
plt.title('Quadratic Curve')
plt.xlabel('X')
plt.ylabel('Y')
plt.grid()
plt.legend()

plt.subplot(2, 3, 3)
plt.plot(x_fine, y_cubic, label='Cubic', color='green')
plt.title('Cubic Curve')
plt.xlabel('X')
plt.ylabel('Y')
plt.grid()
plt.legend()

plt.subplot(2, 3, 4)
plt.plot(x_fine, y_spline, label='Spline', color='red')
plt.title('Spline Curve')
plt.xlabel('X')
plt.ylabel('Y')
plt.grid()
plt.legend()

plt.subplot(2, 3, 5)
plt.plot(x_nurbs[:,0], y_nurbs[:,1], label='NURBS', color='purple')
plt.title('NURBS Curve')
plt.xlabel('X')
plt.ylabel('Y')
plt.grid()
plt.legend()

plt.subplot(2, 3, 6)
plt.plot(x_bezier, y_bezier,label='Bézier', color='brown')
plt.title('Bézier Curve')
plt.xlabel('X')
plt.ylabel('Y')
plt.grid()
plt.legend()

# Adjust layout and show plot
plt.tight_layout()
plt.show()


### A running example

In [ ]:
import openmoc
import numpy as np
import matplotlib.pyplot as plt
import dgl
import torch
import torch.nn as nn
import torch.optim as optim
import bezier  # Importing the bezier library
from scipy import interpolate

# Function to create reactor geometry for 1D or 2D using different curve types including NURBS and Bézier
def create_reactor_geometry(dim, curve_type='linear', control_points=None, weights=None):
    # Define materials
    fuel_material = openmoc.Material(name='Fuel')
    fuel_material.setDensity('g/cm3', 10.0)
    fuel_material.addNuclide('U235', 0.03)

    coolant_material = openmoc.Material(name='Coolant')
    coolant_material.setDensity('g/cm3', 0.7)
    coolant_material.addNuclide('H1', 0.1)

    if dim == 1:
        # Generate points for 1D case
        num_points = 100
        x = np.linspace(0, 10, num_points)

        if curve_type == 'linear':
            y = np.zeros(num_points)  # Linear curve (flat line)
        elif curve_type == 'quadratic':
            y = x**2 / 10  # Quadratic curve (parabola)
        elif curve_type == 'cubic':
            y = (x**3) / 100  # Cubic curve
        elif curve_type == 'spline':
            tck = interpolate.splrep(x, np.sin(x), s=0)  # Spline fit to sine function
            y = interpolate.splev(x, tck)
        elif curve_type == 'nurbs' and control_points is not None and weights is not None:
            # Create a NURBS curve (implementation omitted for brevity)
            pass
        elif curve_type == 'bezier' and control_points is not None:
            # Create a Bézier curve
            nodes = np.array(control_points).T  # Transpose for bezier library format
            curve = bezier.Curve(nodes, degree=len(control_points) - 1)
            x_new = np.linspace(0, 1, num_points)
            y_new = curve.evaluate_list(x_new).T

            x, y = x_new * (10) , y_new[:, 1] * (10)  # Scale to desired range

        else:
            raise ValueError("Unsupported curve type or missing control points for NURBS/Bézier")

        # Create cells along the generated curve points using (x, y)
        cells = []

        for i in range(num_points - 1):
            cell = openmoc.Cell(name=f'Cell {i}')

            cell.setFill(fuel_material if i % 2 == 0 else coolant_material)
            cells.append(cell)

        # Create universe and geometry
        universe = openmoc.Universe(name='Reactor Universe')

        for cell in cells:
            universe.addCell(cell)

        geometry = openmoc.Geometry()
        geometry.addUniverse(universe)

    elif dim == 2:
        # Generate points for 2D case using similar logic as above...
        num_points = 100
        x = np.linspace(0, 10, num_points)

        if curve_type == 'linear':
            y = x
        elif curve_type == 'quadratic':
            y = x**2 / 10
        elif curve_type == 'cubic':
            y = (x**3) / 100
        elif curve_type == 'spline':
            tck = interpolate.splrep(x, np.sin(x), s=0)
            y = interpolate.splev(x, tck)
        elif curve_type == 'nurbs' and control_points is not None and weights is not None:
            # Create a NURBS surface (implementation omitted for brevity)
            pass
        elif curve_type == 'bezier' and control_points is not None:
            # Create a Bézier surface (example with control points)
            nodes2d = np.array(control_points).reshape(-1, 2).T
            surface_degree_u = len(nodes2d[0]) - 1
            surface_degree_v = len(nodes2d[1]) - 1

            bezier_surface_nodes = bezier.Surface(nodes2d, degree=(surface_degree_u, surface_degree_v))

            pts_2d = bezier_surface_nodes.evaluate_list(np.linspace(0, 1, num_points)).T

            x, y = pts_2d[:,0], pts_2d[:,1]

        else:
            raise ValueError("Unsupported curve type or missing control points for NURBS/Bézier")

    return geometry

# Solve NTE using OpenMOC
def solve_nte(geometry):
    solver = openmoc.Solver()

    solver.setGeometry(geometry)
    solver.solve()

    flux = solver.getFlux()

    return flux

# Convert flux results into a graph structure for DGL
def create_graph_from_flux(flux_results):
    num_nodes = len(flux_results)

    src = []
    dst = []

    for i in range(num_nodes):
        if i < num_nodes - 1:
            src.append(i)
            dst.append(i + 1)

        if i > 0:
            src.append(i)
            dst.append(i - 1)

    g = dgl.graph((src, dst))

    g.ndata['flux'] = torch.tensor(flux_results, dtype=torch.float32).view(-1, 1)

    return g

# Define GNN model based on dimensionality
class GNNModel(nn.Module):
    def __init__(self, in_feats, hidden_size, out_feats):
        super(GNNModel, self).__init__()

        self.conv1 = dgl.nn.GraphConv(in_feats, hidden_size)
        self.conv2 = dgl.nn.GraphConv(hidden_size, out_feats)

    def forward(self, g):
        h = g.ndata['flux']

        h = self.conv1(g, h)

        h = torch.relu(h)
        h = self.conv2(g, h)

        return h

# Train the GNN model
def train_gnn(graph):
    model = GNNModel(in_feats=1, hidden_size=16, out_feats=1)

    optimizer = optim.Adam(model.parameters(), lr=0.01)

    for epoch in range(100):
        model.train()

        optimizer.zero_grad()

        output = model(graph)

        loss = ((output - graph.ndata['flux']) ** 2).mean()

        loss.backward()
        optimizer.step()

        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item()}')

# Main execution with user-defined dimensionality and curve type including NURBS and Bézier options.
dimensionality_input = int(input("Select dimensionality (1 or 2): "))
curve_type_input = input("Enter a curve type (linear/quadratic/cubic/spline/nurbs/bezier): ")

control_points_input_str = input("Enter control points as a list of tuples [(x,y), ...]: ")
control_points_input_str_list = eval(control_points_input_str)

weights_input_str_list = input("Enter corresponding weights as a list [w1,w2,...]: ")
weights_input_list_float_str_list= eval(weights_input_str_list)

geometry = create_reactor_geometry(dimensionality_input, curve_type_input,
                                    control_points=control_points_input_str_list,
                                    weights=weights_input_list_float_str_list)

flux_results = solve_nte(geometry)

print("Neutron Flux Results:", flux_results)

# Create graph from flux results and train GNN
graph = create_graph_from_flux(flux_results)
train_gnn(graph)

# Visualization of the generated neutron flux results based on dimensionality selected.
plt.figure(figsize=(12,6))

if dimensionality_input == 1:
    plt.plot(np.linspace(0,10,len(flux_results)), flux_results)
else:
    plt.imshow(flux_results.reshape(10,-1), cmap='hot', interpolation='nearest')

plt.title("Neutron Flux Distribution")
plt.xlabel("Position")
plt.ylabel("Flux")
plt.colorbar(label='Neutron Flux')
plt.show()


## Lets dig deeper into the algorithms

### Bezier interpolation

### Spline interpolation

### B-spline interpolation

### NURBS interpolation

## Reflections